In [1]:
import sys
import os

# Add root directory to Python path
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_dir)

In [2]:
from comparator.config import CONFIG
from comparator.dataset import DatasetManager

dataset = DatasetManager(CONFIG)

# Download YOLOv8 format
yolo_path = dataset.download(data_format="yolov8")

# Or download COCO format
coco_path = dataset.download(data_format="coco")

loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...


In [3]:
from comparator.config import CONFIG
from comparator.trainer import RailTrackModelComparison

# Initialize comparison framework
comparison = RailTrackModelComparison(CONFIG)

# Run complete comparison
comparison.run_comparison()

Using device: mps
Working directory: /Users/milan.jelisavcic/Projects/railguard/notebooks
Data directory: /Users/milan.jelisavcic/Projects/railguard/notebooks/data
Version directory: /Users/milan.jelisavcic/Projects/railguard/notebooks/data/version_5
Starting model comparison for rail track detection...

Training YOLOv8...
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/milan.jelisavcic/Projects/railguard/notebooks/data/version_5/yolov8/data.yaml, epochs=10, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=/Users/milan.jelisavcic/Projects/railguard/notebooks/output, name=yolov8_rail_tracks2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_js

train: Scanning /Users/milan.jelisavcic/Projects/railguard/notebooks/data/version_5/yolov8/train/labels... 6428 images, 18 backgrounds, 0 corrupt: 100%|██████████| 6428/6428 [00:11<00:00, 574.11it/s]


train: New cache created: /Users/milan.jelisavcic/Projects/railguard/notebooks/data/version_5/yolov8/train/labels.cache


val: Scanning /Users/milan.jelisavcic/Projects/railguard/notebooks/data/version_5/yolov8/valid/labels... 307 images, 1 backgrounds, 0 corrupt: 100%|██████████| 307/307 [00:00<00:00, 659.70it/s]

val: New cache created: /Users/milan.jelisavcic/Projects/railguard/notebooks/data/version_5/yolov8/valid/labels.cache


...

WARNING ⚠️ NMS time limit 2.150s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [11:31<00:00, 17.74s/it]


                   all        307        665      0.215     0.0475     0.0362     0.0205

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/10      6.73G      1.354      1.767      1.838          5        640:  67%|██████▋   | 1075/1607 [10:04:01<4:58:55, 33.71s/it]  


KeyboardInterrupt: 